# 🔧 Configuração do Ambiente WSL + VS Code

## ✅ Inserir URL do Servidor Jupyter Manualmente

### Passo 1: Obter a URL do servidor
No terminal WSL que está rodando o Jupyter, procure por uma linha similar a:
```
http://127.0.0.1:8888/lab?token=abc123def456...
```

### Passo 2: Configurar no VS Code
1. **Command Palette** (`Ctrl+Shift+P`)
2. Digite: **"Jupyter: Specify Jupyter Server for Connections"**
3. Selecione **"Existing"**
4. Cole a URL completa com token

### Passo 3: Selecionar o Kernel
1. No notebook, clique no seletor de kernel (canto superior direito)
2. Escolha **"Existing Jupyter Server"**
3. Selecione o servidor que você acabou de configurar

### 🔍 URL Base para WSL:
- **Localhost**: `http://127.0.0.1:8888`
- **WSL IP**: `http://172.x.x.x:8888` (se localhost não funcionar)

### 🛠️ Troubleshooting:
- Se 127.0.0.1 não funcionar, use o IP do WSL: `ip addr show eth0`
- Token está sempre na saída do comando `jupyter lab`
- Porta padrão: 8888

---

In [1]:
# 🌐 Informações de Rede para Conexão Manual
import subprocess
import socket

print("🔍 Informações de rede do WSL:")
print("=" * 50)

# IP do WSL
try:
    result = subprocess.run(['ip', 'addr', 'show', 'eth0'], capture_output=True, text=True)
    lines = result.stdout.split('\n')
    for line in lines:
        if 'inet ' in line and '127.0.0.1' not in line:
            ip = line.strip().split()[1].split('/')[0]
            print(f"📍 IP WSL: {ip}")
            break
except:
    print("❌ Não foi possível obter IP do WSL")

# Portas em uso
print(f"🌐 Localhost: 127.0.0.1")
print(f"🚪 Porta Jupyter padrão: 8888")
print()

print("📋 URLs para testar:")
print("• http://127.0.0.1:8888")
try:
    ip = subprocess.run(['hostname', '-I'], capture_output=True, text=True).stdout.strip().split()[0]
    print(f"• http://{ip}:8888")
except:
    pass

print()
print("💡 Dica: O token aparece na saída do comando 'jupyter lab' no terminal")
print("📝 Formato da URL: http://IP:8888/lab?token=SEU_TOKEN_AQUI")

🔍 Informações de rede do WSL:
📍 IP WSL: 172.29.35.51
🌐 Localhost: 127.0.0.1
🚪 Porta Jupyter padrão: 8888

📋 URLs para testar:
• http://127.0.0.1:8888
• http://172.29.35.51:8888

💡 Dica: O token aparece na saída do comando 'jupyter lab' no terminal
📝 Formato da URL: http://IP:8888/lab?token=SEU_TOKEN_AQUI


In [ ]:
# Teste do ambiente Python
import sys
print(f"🐍 Python executável: {sys.executable}")
print(f"📍 Caminho Python: {sys.path[0]}")
print(f"🔢 Versão Python: {sys.version}")

# Teste simples de importação
try:
    import pandas as pd
    print(f"✅ Pandas importado com sucesso! Versão: {pd.__version__}")
except ImportError as e:
    print(f"❌ Erro ao importar pandas: {e}")
    print(f"? Verifique se o kernel correto está selecionado: 'OBD2 VE Optimizer'")

# Otimização de Mapa VE com IA - Análise de Injeção Eletrônica

Este notebook implementa um modelo de Machine Learning para otimizar mapas de eficiência volumétrica (VE) baseado em dados de sensor O2 wideband.

## Especificações do Motor
- **Cilindrada**: 2000cc
- **Cilindros**: 4
- **Bicos Injetores**: 4 x 19lb/h
- **Modo de Injeção**: Banco a banco (2 injeções por ciclo por banco)

## Objetivos
1. Estimar o delay da sonda O2 no escapamento
2. Analisar erros entre lambda medido vs target
3. Treinar modelo de IA para prever ajustes no mapa VE
4. Gerar nova tabela VE otimizada

## 1. Import Required Libraries and Load Data

In [1]:
# Imports necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import xml.etree.ElementTree as ET
from scipy import interpolate, signal
from scipy.optimize import minimize
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ Bibliotecas importadas com sucesso!")
print(f"📅 Análise iniciada em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Especificações do motor
ENGINE_SPECS = {
    'displacement': 2000,  # cc
    'cylinders': 4,
    'injectors': 4,
    'injector_flow': 19,  # lb/h
    'injection_mode': 'bank_to_bank',
    'stoich_afr': 14.7  # AFR estequiométrico para gasolina
}

print("\n🚗 Especificações do Motor:")
for key, value in ENGINE_SPECS.items():
    print(f"  {key}: {value}")

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Função melhorada para carregar dados do log (incluindo MLG)
import struct

class MLGLogReader:
    """Leitor de arquivos MLG (MegaLogViewer) do TunerStudio"""
    
    def read_mlg_file(self, file_path):
        """Lê arquivo MLG binário e extrai dados"""
        try:
            print(f"📂 Lendo arquivo MLG: {file_path}")
            
            with open(file_path, 'rb') as f:
                content = f.read()
                
                # Procura por padrões de dados conhecidos
                data_start = self._find_data_section(content)
                if data_start == -1:
                    print("❌ Não foi possível encontrar seção de dados")
                    return None
                
                # Extrai dados usando heurística
                df = self._extract_data_heuristic(content, data_start)
                
                if df is not None:
                    print(f"✅ Dados MLG extraídos: {len(df)} registros")
                    return df
                else:
                    print("❌ Falha na extração de dados MLG")
                    return None
                    
        except Exception as e:
            print(f"❌ Erro ao ler MLG: {e}")
            return None
    
    def _find_data_section(self, content):
        """Encontra início da seção de dados numéricos"""
        patterns = [b'Time', b'RPM', b'Lambda', b'TPS']
        
        for pattern in patterns:
            pos = content.find(pattern)
            if pos != -1:
                for i in range(pos, min(pos + 10000, len(content) - 4), 4):
                    try:
                        val = struct.unpack('<f', content[i:i+4])[0]
                        if 500 < val < 8000:  # Possível RPM
                            return i
                    except:
                        continue
        return -1
    
    def _extract_data_heuristic(self, content, start_pos):
        """Extrai dados usando heurística baseada em valores típicos"""
        try:
            estimated_channels = 14
            remaining_bytes = len(content) - start_pos
            record_size = estimated_channels * 4
            num_records = remaining_bytes // record_size
            
            if num_records < 100:
                return None
            
            num_records = min(num_records, 50000)
            
            data_matrix = []
            pos = start_pos
            
            for i in range(num_records):
                if pos + record_size > len(content):
                    break
                
                record = []
                for j in range(estimated_channels):
                    if pos + 4 <= len(content):
                        try:
                            val = struct.unpack('<f', content[pos:pos+4])[0]
                            record.append(val)
                            pos += 4
                        except:
                            record.append(np.nan)
                            pos += 4
                    else:
                        break
                
                if len(record) == estimated_channels:
                    data_matrix.append(record)
            
            columns = ['RPM', 'COOLANT_TEMP', 'MAF', 'THROTTLE_POS', 'INTAKE_TEMP', 
                      'TIMING_ADVANCE', 'ENGINE_LOAD', 'ELM_VOLTAGE', 'SPEED', 
                      'O2_S1_WR_CURRENT', 'O2_S5_WR_CURRENT', 'O2_B2S2', 
                      'SHORT_FUEL_TRIM_1', 'SHORT_FUEL_TRIM_2']
            
            df = pd.DataFrame(data_matrix, columns=columns[:len(data_matrix[0])])
            
            # Filtra valores absurdos
            df = self._clean_extracted_data(df)
            
            return df
            
        except Exception as e:
            print(f"❌ Erro na extração heurística: {e}")
            return None
    
    def _clean_extracted_data(self, df):
        """Limpa dados extraídos removendo valores absurdos"""
        valid_ranges = {
            'RPM': (300, 8000),
            'COOLANT_TEMP': (0, 150),
            'THROTTLE_POS': (0, 100),
            'INTAKE_TEMP': (-20, 80),
            'TIMING_ADVANCE': (-10, 60),
            'ENGINE_LOAD': (0, 150),
            'ELM_VOLTAGE': (10, 16),
            'SPEED': (0, 300),
            'O2_B2S2': (0.4, 1.5)
        }
        
        initial_count = len(df)
        
        for col, (min_val, max_val) in valid_ranges.items():
            if col in df.columns:
                mask = (df[col] >= min_val) & (df[col] <= max_val)
                df = df[mask]
        
        print(f"🧹 Limpeza MLG: {initial_count} -> {len(df)} registros ({len(df)/initial_count*100:.1f}% retidos)")
        
        return df.reset_index(drop=True)

def load_ecu_log_data_advanced(file_path):
    """Carrega e processa dados do log da ECU com filtros de qualidade"""
    try:
        # Determina tipo de arquivo
        if file_path.endswith('.mlg'):
            reader = MLGLogReader()
            df = reader.read_mlg_file(file_path)
            if df is None:
                return None
        else:
            # Arquivo CSV/texto
            columns = ['RPM', 'COOLANT_TEMP', 'MAF', 'THROTTLE_POS', 'INTAKE_TEMP', 
                      'TIMING_ADVANCE', 'ENGINE_LOAD', 'ELM_VOLTAGE', 'SPEED', 
                      'O2_S1_WR_CURRENT', 'O2_S5_WR_CURRENT', 'O2_B2S2', 
                      'SHORT_FUEL_TRIM_1', 'SHORT_FUEL_TRIM_2']
            
            df = pd.read_csv(file_path, names=columns)
            print(f"📊 Log texto carregado: {len(df)} registros")
        
        # Adiciona campos calculados
        df['Time'] = np.arange(len(df)) * 0.1
        df['Lambda'] = df['O2_B2S2'] / ENGINE_SPECS['stoich_afr']
        df['FuelLoad'] = df['ENGINE_LOAD']
        
        # Taxa de mudança do TPS (suavizada)
        df['TPS_Rate'] = df['THROTTLE_POS'].diff() / 0.1
        df['TPS_Rate'] = df['TPS_Rate'].rolling(window=3, center=True).mean()
        df['TPS_Rate'] = df['TPS_Rate'].fillna(0)
        
        # Estimativa de PW (pulso dos bicos)
        df['PW_Estimated'] = (df['ENGINE_LOAD'] * df['RPM'] * 2.0) / (19 * 14.7 * 10000)
        df['PW_Estimated'] = np.clip(df['PW_Estimated'], 0, 20)
        
        # Detecta fuel cut (DFCO)
        df['Is_FuelCut'] = (df['PW_Estimated'] < 1.0) | (df['Lambda'] > 1.1)
        
        print(f"? Campos calculados adicionados")
        
        return df
        
    except Exception as e:
        print(f"❌ Erro ao carregar log: {e}")
        return None

# Testa carregamento com arquivo MLG maior
mlg_file = '/home/robca/obd2/logs/2025-08-06_00.09.39.mlg'
csv_file = '/home/robca/obd2/logs/example.log'

print("🔍 Testando leitura de arquivos...")

# Testa MLG
df_mlg = load_ecu_log_data_advanced(mlg_file)
if df_mlg is not None:
    print(f"\n📊 Dados MLG - Estatísticas básicas:")
    print(df_mlg[['RPM', 'Lambda', 'FuelLoad', 'THROTTLE_POS']].describe().round(3))

# Testa CSV (referência)
df_csv = load_ecu_log_data_advanced(csv_file)
if df_csv is not None:
    print(f"\n? Dados CSV - Estatísticas básicas:")
    print(df_csv[['RPM', 'Lambda', 'FuelLoad', 'THROTTLE_POS']].describe().round(3))

# Escolhe o melhor dataset
if df_mlg is not None and len(df_mlg) > len(df_csv) if df_csv is not None else 0:
    df_log = df_mlg
    print(f"\n✅ Usando dados MLG ({len(df_log)} registros)")
elif df_csv is not None:
    df_log = df_csv
    print(f"\n✅ Usando dados CSV ({len(df_log)} registros)")
else:
    df_log = None
    print(f"\n❌ Nenhum arquivo válido encontrado")

In [ ]:
# Filtros de Qualidade para Melhor Precisão
def apply_quality_filters(df, tps_threshold=2.0, lambda_range=(0.7, 1.3), min_pw=1.5):
    """Aplica filtros de qualidade conforme solicitado pelo usuário"""
    if df is None or len(df) == 0:
        return None
    
    initial_count = len(df)
    print(f"🎯 Aplicando filtros de qualidade...")
    print(f"📊 Registros iniciais: {initial_count}")
    
    # 1. TPS estável (mudança < threshold %/segundo)
    tps_stable = abs(df['TPS_Rate']) <= tps_threshold
    df = df[tps_stable]
    print(f"🎛️ Após filtro TPS estável (±{tps_threshold}%/s): {len(df)} registros ({len(df)/initial_count*100:.1f}%)")
    
    # 2. Exclui momentos de fuel cut
    no_fuel_cut = ~df['Is_FuelCut']
    df = df[no_fuel_cut]
    print(f"⛽ Após exclusão de fuel cut: {len(df)} registros ({len(df)/initial_count*100:.1f}%)")
    
    # 3. Lambda válido (range típico)
    lambda_valid = (df['Lambda'] >= lambda_range[0]) & (df['Lambda'] <= lambda_range[1])
    df = df[lambda_valid]
    print(f"🌡️ Após filtro lambda ({lambda_range[0]}-{lambda_range[1]}): {len(df)} registros ({len(df)/initial_count*100:.1f}%)")
    
    # 4. RPM e carga mínimos para análise confiável
    operating_range = (df['RPM'] >= 1000) & (df['FuelLoad'] >= 20)
    df = df[operating_range]
    print(f"🔧 Após filtro de range operacional: {len(df)} registros ({len(df)/initial_count*100:.1f}%)")
    
    if len(df) < 100:
        print(f"⚠️ Poucos dados após filtros ({len(df)}). Considere relaxar critérios.")
        return None
    
    print(f"✅ Filtros aplicados: {len(df)} registros finais ({len(df)/initial_count*100:.1f}% dos dados originais)")
    
    return df.reset_index(drop=True)

def analyze_tps_stability(df, window=50):
    """Analisa estabilidade do TPS para determinar threshold ideal"""
    if df is None:
        return
    
    print(f"📊 Análise de estabilidade do TPS:")
    print(f"TPS Rate - Min: {df['TPS_Rate'].min():.2f}, Max: {df['TPS_Rate'].max():.2f}")
    print(f"TPS Rate - Média: {df['TPS_Rate'].mean():.2f}, Std: {df['TPS_Rate'].std():.2f}")
    
    # Percentis para entender distribuição
    percentiles = [50, 75, 90, 95, 99]
    tps_rate_abs = abs(df['TPS_Rate'])
    
    print(f"\nPercentis de |TPS_Rate| (%/s):")
    for p in percentiles:
        val = np.percentile(tps_rate_abs, p)
        count = sum(tps_rate_abs <= val)
        pct = count / len(df) * 100
        print(f"  P{p}: {val:.2f} ({pct:.1f}% dos dados)")
    
    # Visualização da distribuição
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Histograma de TPS Rate
    ax1.hist(df['TPS_Rate'], bins=50, alpha=0.7, edgecolor='black')
    ax1.axvline(0, color='red', linestyle='--', label='Zero')
    ax1.axvline(-2, color='orange', linestyle='--', label='±2%/s')
    ax1.axvline(2, color='orange', linestyle='--')
    ax1.set_xlabel('TPS Rate (%/s)')
    ax1.set_ylabel('Frequência')
    ax1.set_title('Distribuição da Taxa de Mudança do TPS')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # TPS ao longo do tempo (amostra)
    sample_size = min(1000, len(df))
    sample_idx = np.linspace(0, len(df)-1, sample_size, dtype=int)
    
    ax2.plot(df.iloc[sample_idx]['Time'], df.iloc[sample_idx]['THROTTLE_POS'], 'b-', alpha=0.7, label='TPS')
    ax2.set_xlabel('Tempo (s)')
    ax2.set_ylabel('TPS (%)')
    ax2.set_title('TPS ao Longo do Tempo (amostra)')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

# Aplica filtros se temos dados
if df_log is not None:
    analyze_tps_stability(df_log)
    df_filtered = apply_quality_filters(df_log, tps_threshold=2.0)
    
    if df_filtered is not None:
        print(f"\n🎯 Comparação antes/depois dos filtros:")
        print(f"Lambda - Original: μ={df_log['Lambda'].mean():.3f}, σ={df_log['Lambda'].std():.3f}")
        print(f"Lambda - Filtrado: μ={df_filtered['Lambda'].mean():.3f}, σ={df_filtered['Lambda'].std():.3f}")
        print(f"TPS Rate - Original: μ={abs(df_log['TPS_Rate']).mean():.3f}")
        print(f"TPS Rate - Filtrado: μ={abs(df_filtered['TPS_Rate']).mean():.3f}")
else:
    print("❌ Sem dados para aplicar filtros")

In [ ]:
# Função para carregar tabela VE
def load_ve_table(file_path):
    """Carrega tabela VE do arquivo XML"""
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        
        # Namespace
        ns = {'table': 'http://www.EFIAnalytics.com/:table'}
        
        # Eixo X (RPM)
        x_axis = root.find('.//table:xAxis', ns)
        rpm_bins = [float(x) for x in x_axis.text.strip().split()]
        
        # Eixo Y (FuelLoad)
        y_axis = root.find('.//table:yAxis', ns)
        load_bins = [float(x) for x in y_axis.text.strip().split()]
        
        # Valores VE
        z_values = root.find('.//table:zValues', ns)
        ve_values = [float(x) for x in z_values.text.strip().split()]
        
        # Converte para matriz
        ve_matrix = np.array(ve_values).reshape(len(load_bins), len(rpm_bins))
        
        print(f"🗂️  Tabela VE carregada: {len(load_bins)}x{len(rpm_bins)} células")
        print(f"📊 RPM range: {min(rpm_bins):.0f} - {max(rpm_bins):.0f}")
        print(f"📊 Load range: {min(load_bins):.0f} - {max(load_bins):.0f} kPa")
        print(f"📊 VE range: {ve_matrix.min():.1f}% - {ve_matrix.max():.1f}%")
        
        return rpm_bins, load_bins, ve_matrix
        
    except Exception as e:
        print(f"❌ Erro ao carregar tabela VE: {e}")
        return None, None, None

# Carrega tabela VE
ve_file = '/home/robca/obd2/logs/ve.table'
rpm_bins, load_bins, ve_table = load_ve_table(ve_file)

# Visualiza a tabela VE original
if ve_table is not None:
    plt.figure(figsize=(14, 8))
    
    # Heatmap da tabela VE
    plt.subplot(1, 2, 1)
    im = plt.imshow(ve_table, aspect='auto', origin='lower', cmap='RdYlBu_r')
    plt.colorbar(im, label='VE (%)')
    plt.title('Tabela VE Original')
    plt.xlabel('RPM Bins')
    plt.ylabel('Load Bins (kPa)')
    
    # Configurar ticks
    rpm_ticks = np.arange(0, len(rpm_bins), 2)
    load_ticks = np.arange(0, len(load_bins), 2)
    plt.xticks(rpm_ticks, [f'{rpm_bins[i]:.0f}' for i in rpm_ticks], rotation=45)
    plt.yticks(load_ticks, [f'{load_bins[i]:.0f}' for i in load_ticks])
    
    # Histograma dos valores VE
    plt.subplot(1, 2, 2)
    plt.hist(ve_table.flatten(), bins=30, alpha=0.7, edgecolor='black')
    plt.xlabel('VE (%)')
    plt.ylabel('Frequência')
    plt.title('Distribuição dos Valores VE')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 2. Data Preprocessing and Cleaning

In [ ]:
# Análise Exploratória dos Dados com Filtros de Qualidade

# Usa dados filtrados se disponível, senão usa originais
if 'df_filtered' in locals() and df_filtered is not None:
    df_analysis = df_filtered
    print("🎯 Usando dados com filtros de qualidade aplicados")
else:
    df_analysis = df_log
    print("📊 Usando dados originais (sem filtros)")

if df_analysis is None:
    print("❌ Nenhum dado disponível para análise")
else:
    print(f"📈 Analisando {len(df_analysis)} registros")
    
    # Estatísticas descritivas
    print(f"\n📊 Estatísticas dos principais parâmetros:")
    key_columns = ['RPM', 'Lambda', 'FuelLoad', 'THROTTLE_POS', 'TPS_Rate']
    stats = df_analysis[key_columns].describe()
    print(stats.round(3))
    
    # Visualizações melhoradas
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    # 1. Distribuição Lambda
    axes[0,0].hist(df_analysis['Lambda'], bins=50, alpha=0.7, color='blue', edgecolor='black')
    axes[0,0].axvline(1.0, color='red', linestyle='--', label='Stoich (λ=1.0)')
    axes[0,0].set_xlabel('Lambda')
    axes[0,0].set_ylabel('Frequência')
    axes[0,0].set_title('Distribuição Lambda')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. Lambda vs RPM
    scatter = axes[0,1].scatter(df_analysis['RPM'], df_analysis['Lambda'], 
                               c=df_analysis['FuelLoad'], cmap='viridis', alpha=0.6, s=1)
    axes[0,1].axhline(1.0, color='red', linestyle='--', label='Stoich')
    axes[0,1].set_xlabel('RPM')
    axes[0,1].set_ylabel('Lambda')
    axes[0,1].set_title('Lambda vs RPM (cor = FuelLoad)')
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=axes[0,1], label='FuelLoad')
    
    # 3. TPS Rate (novo)
    axes[0,2].hist(abs(df_analysis['TPS_Rate']), bins=50, alpha=0.7, color='orange', edgecolor='black')
    axes[0,2].axvline(2.0, color='red', linestyle='--', label='Threshold (2%/s)')
    axes[0,2].set_xlabel('|TPS Rate| (%/s)')
    axes[0,2].set_ylabel('Frequência')
    axes[0,2].set_title('Distribuição |TPS Rate|')
    axes[0,2].legend()
    axes[0,2].grid(True, alpha=0.3)
    
    # 4. Lambda vs FuelLoad
    scatter2 = axes[1,0].scatter(df_analysis['FuelLoad'], df_analysis['Lambda'], 
                                c=df_analysis['RPM'], cmap='plasma', alpha=0.6, s=1)
    axes[1,0].axhline(1.0, color='red', linestyle='--', label='Stoich')
    axes[1,0].set_xlabel('FuelLoad')
    axes[1,0].set_ylabel('Lambda')
    axes[1,0].set_title('Lambda vs FuelLoad (cor = RPM)')
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)
    plt.colorbar(scatter2, ax=axes[1,0], label='RPM')
    
    # 5. Timeline Lambda (amostra)
    sample_size = min(2000, len(df_analysis))
    sample_idx = np.linspace(0, len(df_analysis)-1, sample_size, dtype=int)
    sample_data = df_analysis.iloc[sample_idx]
    
    axes[1,1].plot(sample_data['Time'], sample_data['Lambda'], 'b-', alpha=0.7, linewidth=0.5)
    axes[1,1].axhline(1.0, color='red', linestyle='--', label='Stoich')
    axes[1,1].set_xlabel('Tempo (s)')
    axes[1,1].set_ylabel('Lambda')
    axes[1,1].set_title('Lambda ao Longo do Tempo')
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)
    
    # 6. Correlação com TPS
    axes[1,2].scatter(df_analysis['THROTTLE_POS'], df_analysis['Lambda'], 
                     c=df_analysis['TPS_Rate'], cmap='RdYlBu', alpha=0.6, s=1)
    axes[1,2].axhline(1.0, color='red', linestyle='--', label='Stoich')
    axes[1,2].set_xlabel('TPS (%)')
    axes[1,2].set_ylabel('Lambda')
    axes[1,2].set_title('Lambda vs TPS (cor = TPS Rate)')
    axes[1,2].legend()
    axes[1,2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Análise de correlações
    print(f"\n🔗 Matriz de correlações:")
    corr_columns = ['RPM', 'Lambda', 'FuelLoad', 'THROTTLE_POS', 'TPS_Rate', 'INTAKE_TEMP']
    correlations = df_analysis[corr_columns].corr()
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(correlations, annot=True, cmap='RdBu_r', center=0, 
                square=True, fmt='.3f')
    plt.title('Matriz de Correlações')
    plt.tight_layout()
    plt.show()
    
    # Qualidade dos dados
    print(f"\n✅ Qualidade dos dados filtrados:")
    print(f"• Range Lambda: {df_analysis['Lambda'].min():.3f} - {df_analysis['Lambda'].max():.3f}")
    print(f"• Std Lambda: {df_analysis['Lambda'].std():.3f}")
    print(f"• % dados com TPS estável: {(abs(df_analysis['TPS_Rate']) <= 2.0).mean()*100:.1f}%")
    print(f"• % dados sem fuel cut: {(~df_analysis['Is_FuelCut']).mean()*100:.1f}%")

## 3. Lambda Sensor Delay Analysis

In [ ]:
# Função para estimar delay da sonda O2
def estimate_lambda_sensor_delay(df):
    """Estima o delay da sonda lambda baseado em eventos de transiente"""
    
    print("🔍 Analisando delay da sonda lambda...")
    
    # 1. Detecta eventos de mudança rápida no TPS (aceleração/desaceleração)
    tps_threshold = 30  # %/s
    rapid_tps_events = df[abs(df['TPS_Rate']) > tps_threshold].copy()
    
    print(f"📈 Eventos rápidos de TPS detectados: {len(rapid_tps_events)}")
    
    # 2. Detecta cut-off de combustível (DFCO) - simulado por queda abrupta em lambda
    # Identifica pontos onde lambda aumenta rapidamente (corte de combustível)
    lambda_spike_threshold = 0.1  # lambda/s
    fuel_cut_events = df[df['Lambda_Rate'] > lambda_spike_threshold].copy()
    
    print(f"⛽ Eventos de corte de combustível detectados: {len(fuel_cut_events)}")
    
    # 3. Analisa correlação cruzada para estimar delay
    delays = []
    
    # Para cada evento de TPS rápido
    for idx in rapid_tps_events.index[:20]:  # Analisa até 20 eventos
        if idx + 100 < len(df):  # Garante janela de análise
            
            # Janela de análise (10 segundos)
            window_start = max(0, idx - 50)
            window_end = min(len(df), idx + 100)
            window = df.iloc[window_start:window_end].copy()
            
            # Sinais para correlação
            tps_signal = window['TPS_Rate'].values
            lambda_signal = window['Lambda_Rate'].values
            
            # Correlação cruzada
            correlation = np.correlate(lambda_signal, tps_signal, mode='full')
            
            # Encontra o lag com maior correlação
            lag_samples = np.argmax(np.abs(correlation)) - len(tps_signal) + 1
            
            # Converte para tempo (segundos)
            lag_time = lag_samples * 0.1
            
            # Filtra delays plausíveis (0.1 a 2.0 segundos)
            if 0.1 <= lag_time <= 2.0:
                delays.append(lag_time)
    
    # Estatísticas dos delays
    if delays:
        estimated_delay = np.median(delays)
        delay_std = np.std(delays)
        delay_samples = int(estimated_delay / 0.1)
        
        print(f"⏱️  Delay estimado: {estimated_delay:.2f}s ± {delay_std:.2f}s")
        print(f"📊 Baseado em {len(delays)} eventos válidos")
        print(f"🔢 Delay em amostras: {delay_samples}")
        
        # Visualiza análise de delay
        plt.figure(figsize=(15, 10))
        
        # Histograma dos delays
        plt.subplot(2, 3, 1)
        plt.hist(delays, bins=20, alpha=0.7, edgecolor='black')
        plt.axvline(estimated_delay, color='red', linestyle='--', label=f'Mediana: {estimated_delay:.2f}s')
        plt.xlabel('Delay (segundos)')
        plt.ylabel('Frequência')
        plt.title('Distribuição dos Delays Estimados')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Exemplo de evento
        if len(rapid_tps_events) > 0:
            example_idx = rapid_tps_events.index[0]
            window_start = max(0, example_idx - 50)
            window_end = min(len(df), example_idx + 100)
            example_window = df.iloc[window_start:window_end].copy()
            
            plt.subplot(2, 3, 2)
            plt.plot(example_window['Time'], example_window['THROTTLE_POS'], label='TPS (%)', linewidth=2)
            plt.axvline(df.iloc[example_idx]['Time'], color='red', linestyle='--', alpha=0.7, label='Evento TPS')
            plt.xlabel('Tempo (s)')
            plt.ylabel('TPS (%)')\n            plt.title('Exemplo: Evento de TPS')\n            plt.legend()\n            plt.grid(True, alpha=0.3)\n            \n            plt.subplot(2, 3, 3)\n            plt.plot(example_window['Time'], example_window['Lambda'], label='Lambda', linewidth=2, color='orange')\n            plt.axvline(df.iloc[example_idx]['Time'], color='red', linestyle='--', alpha=0.7, label='Evento TPS')\n            plt.axvline(df.iloc[example_idx]['Time'] + estimated_delay, color='green', \n                       linestyle='--', alpha=0.7, label=f'Lambda Response (+{estimated_delay:.2f}s)')\n            plt.xlabel('Tempo (s)')\n            plt.ylabel('Lambda')\n            plt.title('Resposta do Lambda')\n            plt.legend()\n            plt.grid(True, alpha=0.3)\n        \n        # Análise de eventos de TPS\n        plt.subplot(2, 3, 4)\n        plt.scatter(df['Time'], df['TPS_Rate'], alpha=0.3, s=1, label='TPS Rate')\n        plt.scatter(rapid_tps_events['Time'], rapid_tps_events['TPS_Rate'], \n                   color='red', s=20, label=f'Eventos Rápidos (>{tps_threshold}%/s)')\n        plt.xlabel('Tempo (s)')\n        plt.ylabel('TPS Rate (%/s)')\n        plt.title('Detecção de Eventos de TPS')\n        plt.legend()\n        plt.grid(True, alpha=0.3)\n        \n        # Análise de eventos de lambda\n        plt.subplot(2, 3, 5)\n        plt.scatter(df['Time'], df['Lambda_Rate'], alpha=0.3, s=1, label='Lambda Rate')\n        plt.scatter(fuel_cut_events['Time'], fuel_cut_events['Lambda_Rate'], \n                   color='orange', s=20, label=f'Fuel Cut (>{lambda_spike_threshold}/s)')\n        plt.xlabel('Tempo (s)')\n        plt.ylabel('Lambda Rate (/s)')\n        plt.title('Detecção de Eventos de Lambda')\n        plt.legend()\n        plt.grid(True, alpha=0.3)\n        \n        # Correlação TPS vs Lambda com delay\n        plt.subplot(2, 3, 6)\n        # Aplica delay ao lambda\n        lambda_delayed = df['Lambda'].shift(delay_samples)\n        valid_mask = ~(df['TPS_Rate'].isna() | lambda_delayed.isna())\n        plt.scatter(df[valid_mask]['TPS_Rate'], lambda_delayed[valid_mask], alpha=0.5, s=1)\n        plt.xlabel('TPS Rate (%/s)')\n        plt.ylabel(f'Lambda (delayed +{estimated_delay:.2f}s)')\n        plt.title('Correlação TPS vs Lambda (com delay)')\n        plt.grid(True, alpha=0.3)\n        \n        plt.tight_layout()\n        plt.show()\n        \n        return estimated_delay, delay_samples, delays\n        \n    else:\n        print(\"❌ Não foi possível estimar delay da sonda\")\n        return 0.0, 0, []\n\n# Estima delay da sonda\nif df_processed is not None:\n    estimated_delay, delay_samples, delay_list = estimate_lambda_sensor_delay(df_processed)\nelse:\n    estimated_delay, delay_samples = 0.0, 0

## 4. VE Table Current vs Target Analysis

In [ ]:
# Funções para análise VE vs Target
def interpolate_ve_current(rpm, fuel_load, rpm_bins, load_bins, ve_table):
    """Interpola VE atual da tabela para RPM e Load específicos"""
    if ve_table is None:
        return 50.0
    
    # Interpolação bilinear
    interp_func = interpolate.interp2d(rpm_bins, load_bins, ve_table, kind='linear')
    
    # Limita valores aos ranges da tabela
    rpm_clipped = np.clip(rpm, min(rpm_bins), max(rpm_bins))
    load_clipped = np.clip(fuel_load, min(load_bins), max(load_bins))
    
    return float(interp_func(rpm_clipped, load_clipped)[0])

def calculate_lambda_target(rpm, fuel_load):
    """Calcula lambda target baseado em condições de operação"""
    # Estratégia de lambda target:\n    # - Baixa carga (cruise): λ ≈ 1.0 (estequiométrico)\n    # - Carga média (aceleração): λ ≈ 0.95\n    # - Alta carga (WOT): λ ≈ 0.85-0.87 (proteção)\n    \n    if fuel_load < 25:\n        return 1.0  # Cruise - economia\n    elif fuel_load < 60:\n        return 0.95  # Aceleração moderada\n    elif fuel_load < 80:\n        return 0.88  # Aceleração forte\n    else:\n        return 0.85  # WOT - proteção\n\n# Análise dos dados com VE e targets\ndef analyze_ve_vs_target(df, rpm_bins, load_bins, ve_table, delay_samples=0):\n    \"\"\"Analisa VE atual vs lambda target e calcula correções necessárias\"\"\"\n    \n    print(\"🔍 Analisando VE atual vs Lambda target...\")\n    \n    # Cria cópia dos dados\n    df_analysis = df.copy()\n    \n    # Interpola VE atual para cada ponto\n    df_analysis['VE_Current'] = df_analysis.apply(\n        lambda row: interpolate_ve_current(row['RPM'], row['FuelLoad'], rpm_bins, load_bins, ve_table),\n        axis=1\n    )\n    \n    # Calcula lambda target para cada ponto\n    df_analysis['Lambda_Target'] = df_analysis.apply(\n        lambda row: calculate_lambda_target(row['RPM'], row['FuelLoad']),\n        axis=1\n    )\n    \n    # Aplica delay ao lambda se estimado\n    if delay_samples > 0:\n        df_analysis['Lambda_Delayed'] = df_analysis['Lambda'].shift(-delay_samples)\n        # Remove pontos sem lambda delayed\n        df_analysis = df_analysis[:-delay_samples] if delay_samples > 0 else df_analysis\n    else:\n        df_analysis['Lambda_Delayed'] = df_analysis['Lambda']\n    \n    # Remove NaN\n    df_analysis = df_analysis.dropna()\n    \n    # Calcula erros e correções\n    df_analysis['Lambda_Error'] = df_analysis['Lambda_Delayed'] - df_analysis['Lambda_Target']\n    df_analysis['Lambda_Error_Percent'] = (df_analysis['Lambda_Error'] / df_analysis['Lambda_Target']) * 100\n    \n    # Calcula fator de correção VE\n    # Se lambda é maior que target (mistura pobre), precisa mais combustível (VE maior)\n    df_analysis['VE_Correction_Factor'] = df_analysis['Lambda_Target'] / df_analysis['Lambda_Delayed']\n    df_analysis['VE_Corrected'] = df_analysis['VE_Current'] * df_analysis['VE_Correction_Factor']\n    \n    # Limita correções extremas (±20%)\n    df_analysis['VE_Corrected'] = np.clip(\n        df_analysis['VE_Corrected'],\n        df_analysis['VE_Current'] * 0.8,\n        df_analysis['VE_Current'] * 1.2\n    )\n    \n    df_analysis['VE_Change_Percent'] = ((df_analysis['VE_Corrected'] - df_analysis['VE_Current']) / \n                                       df_analysis['VE_Current']) * 100\n    \n    print(f\"📊 Dados para análise: {len(df_analysis)} pontos\")\n    print(f\"⚖️  Erro médio de lambda: {df_analysis['Lambda_Error'].mean():.3f}\")\n    print(f\"📈 Correção média de VE: {df_analysis['VE_Change_Percent'].mean():.1f}%\")\n    \n    return df_analysis\n\n# Executa análise\nif df_processed is not None and ve_table is not None:\n    df_ve_analysis = analyze_ve_vs_target(df_processed, rpm_bins, load_bins, ve_table, delay_samples)\n    \n    # Visualizações da análise\n    fig, axes = plt.subplots(3, 3, figsize=(20, 15))\n    \n    # 1. Lambda atual vs target\n    axes[0,0].scatter(df_ve_analysis['Lambda_Target'], df_ve_analysis['Lambda_Delayed'], alpha=0.5, s=1)\n    axes[0,0].plot([0.8, 1.1], [0.8, 1.1], 'r--', linewidth=2, label='Ideal (1:1)')\n    axes[0,0].set_xlabel('Lambda Target')\n    axes[0,0].set_ylabel('Lambda Medido (delayed)')\n    axes[0,0].set_title('Lambda Medido vs Target')\n    axes[0,0].legend()\n    axes[0,0].grid(True, alpha=0.3)\n    \n    # 2. Erro de lambda por RPM\n    axes[0,1].scatter(df_ve_analysis['RPM'], df_ve_analysis['Lambda_Error'], alpha=0.5, s=1, c=df_ve_analysis['FuelLoad'], cmap='viridis')\n    axes[0,1].axhline(y=0, color='red', linestyle='--', alpha=0.7)\n    axes[0,1].set_xlabel('RPM')\n    axes[0,1].set_ylabel('Erro Lambda')\n    axes[0,1].set_title('Erro Lambda por RPM (cor = FuelLoad)')\n    axes[0,1].grid(True, alpha=0.3)\n    \n    # 3. Erro de lambda por carga\n    axes[0,2].scatter(df_ve_analysis['FuelLoad'], df_ve_analysis['Lambda_Error'], alpha=0.5, s=1, c=df_ve_analysis['RPM'], cmap='plasma')\n    axes[0,2].axhline(y=0, color='red', linestyle='--', alpha=0.7)\n    axes[0,2].set_xlabel('Fuel Load (kPa)')\n    axes[0,2].set_ylabel('Erro Lambda')\n    axes[0,2].set_title('Erro Lambda por Carga (cor = RPM)')\n    axes[0,2].grid(True, alpha=0.3)\n    \n    # 4. VE atual vs corrigido\n    axes[1,0].scatter(df_ve_analysis['VE_Current'], df_ve_analysis['VE_Corrected'], alpha=0.5, s=1)\n    min_ve = min(df_ve_analysis['VE_Current'].min(), df_ve_analysis['VE_Corrected'].min())\n    max_ve = max(df_ve_analysis['VE_Current'].max(), df_ve_analysis['VE_Corrected'].max())\n    axes[1,0].plot([min_ve, max_ve], [min_ve, max_ve], 'r--', linewidth=2, label='Sem mudança')\n    axes[1,0].set_xlabel('VE Atual (%)')\n    axes[1,0].set_ylabel('VE Corrigido (%)')\n    axes[1,0].set_title('VE Atual vs Corrigido')\n    axes[1,0].legend()\n    axes[1,0].grid(True, alpha=0.3)\n    \n    # 5. Correção VE por RPM\n    axes[1,1].scatter(df_ve_analysis['RPM'], df_ve_analysis['VE_Change_Percent'], alpha=0.5, s=1, c=df_ve_analysis['FuelLoad'], cmap='viridis')\n    axes[1,1].axhline(y=0, color='red', linestyle='--', alpha=0.7)\n    axes[1,1].set_xlabel('RPM')\n    axes[1,1].set_ylabel('Mudança VE (%)')\n    axes[1,1].set_title('Correção VE por RPM')\n    axes[1,1].grid(True, alpha=0.3)\n    \n    # 6. Correção VE por carga\n    axes[1,2].scatter(df_ve_analysis['FuelLoad'], df_ve_analysis['VE_Change_Percent'], alpha=0.5, s=1, c=df_ve_analysis['RPM'], cmap='plasma')\n    axes[1,2].axhline(y=0, color='red', linestyle='--', alpha=0.7)\n    axes[1,2].set_xlabel('Fuel Load (kPa)')\n    axes[1,2].set_ylabel('Mudança VE (%)')\n    axes[1,2].set_title('Correção VE por Carga')\n    axes[1,2].grid(True, alpha=0.3)\n    \n    # 7. Histograma de erros lambda\n    axes[2,0].hist(df_ve_analysis['Lambda_Error'], bins=50, alpha=0.7, edgecolor='black')\n    axes[2,0].axvline(0, color='red', linestyle='--', alpha=0.7)\n    axes[2,0].set_xlabel('Erro Lambda')\n    axes[2,0].set_ylabel('Frequência')\n    axes[2,0].set_title('Distribuição do Erro Lambda')\n    axes[2,0].grid(True, alpha=0.3)\n    \n    # 8. Histograma de correções VE\n    axes[2,1].hist(df_ve_analysis['VE_Change_Percent'], bins=50, alpha=0.7, edgecolor='black')\n    axes[2,1].axvline(0, color='red', linestyle='--', alpha=0.7)\n    axes[2,1].set_xlabel('Mudança VE (%)')\n    axes[2,1].set_ylabel('Frequência')\n    axes[2,1].set_title('Distribuição das Correções VE')\n    axes[2,1].grid(True, alpha=0.3)\n    \n    # 9. Correlação entre erro lambda e correção VE\n    axes[2,2].scatter(df_ve_analysis['Lambda_Error'], df_ve_analysis['VE_Change_Percent'], alpha=0.5, s=1)\n    axes[2,2].set_xlabel('Erro Lambda')\n    axes[2,2].set_ylabel('Mudança VE (%)')\n    axes[2,2].set_title('Correlação: Erro Lambda vs Correção VE')\n    axes[2,2].grid(True, alpha=0.3)\n    \n    plt.tight_layout()\n    plt.show()\n    \n    # Estatísticas por zona de operação\n    print(\"\\n📊 Estatísticas por zona de operação:\")\n    \n    # Divide em zonas baseadas em carga\n    zones = [\n        (df_ve_analysis['FuelLoad'] < 30, 'Baixa Carga'),\n        ((df_ve_analysis['FuelLoad'] >= 30) & (df_ve_analysis['FuelLoad'] < 70), 'Carga Média'),\n        (df_ve_analysis['FuelLoad'] >= 70, 'Alta Carga')\n    ]\n    \n    for mask, zone_name in zones:\n        zone_data = df_ve_analysis[mask]\n        if len(zone_data) > 0:\n            print(f\"\\n{zone_name}:\")\n            print(f\"  Pontos: {len(zone_data)}\")\n            print(f\"  Erro lambda médio: {zone_data['Lambda_Error'].mean():.3f} ± {zone_data['Lambda_Error'].std():.3f}\")\n            print(f\"  Correção VE média: {zone_data['VE_Change_Percent'].mean():.1f}% ± {zone_data['VE_Change_Percent'].std():.1f}%\")\n            print(f\"  Range RPM: {zone_data['RPM'].min():.0f} - {zone_data['RPM'].max():.0f}\")\nelse:\n    print(\"❌ Dados não disponíveis para análise VE\")\n    df_ve_analysis = None

## 5. Feature Engineering for Prediction Model

In [ ]:
# Feature Engineering para modelo de predição\ndef create_features_for_ml(df_analysis):\n    \"\"\"Cria features para treinamento do modelo de ML\"\"\"\n    \n    print(\"🔧 Criando features para modelo de ML...\")\n    \n    df_features = df_analysis.copy()\n    \n    # 1. Features básicas já existentes\n    basic_features = ['RPM', 'FuelLoad', 'VE_Current', 'Lambda_Delayed', 'Lambda_Target',\n                     'COOLANT_TEMP', 'INTAKE_TEMP', 'TIMING_ADVANCE']\n    \n    # 2. Features de interação\n    df_features['RPM_Load_Interaction'] = df_features['RPM'] * df_features['FuelLoad'] / 1000\n    df_features['VE_Lambda_Interaction'] = df_features['VE_Current'] * df_features['Lambda_Delayed']\n    \n    # 3. Features categóricas baseadas em ranges\n    # Range de RPM\n    df_features['RPM_Range'] = pd.cut(df_features['RPM'], \n                                     bins=[0, 1500, 3000, 5000, 8000], \n                                     labels=['Idle', 'Low', 'Mid', 'High'])\n    \n    # Range de Carga\n    df_features['Load_Range'] = pd.cut(df_features['FuelLoad'], \n                                      bins=[0, 30, 70, 100, 150], \n                                      labels=['Light', 'Moderate', 'Heavy', 'WOT'])\n    \n    # 4. Features temporais (tendências)\n    # Suavização para reduzir ruído\n    window = 5\n    df_features['RPM_Trend'] = df_features['RPM'].rolling(window).mean().fillna(df_features['RPM'])\n    df_features['Load_Trend'] = df_features['FuelLoad'].rolling(window).mean().fillna(df_features['FuelLoad'])\n    df_features['Lambda_Trend'] = df_features['Lambda_Delayed'].rolling(window).mean().fillna(df_features['Lambda_Delayed'])\n    \n    # 5. Features de eficiência\n    # Eficiência atual (quão longe do target)\n    df_features['Lambda_Efficiency'] = 1 - abs(df_features['Lambda_Error']) / df_features['Lambda_Target']\n    \n    # 6. Features físicas do motor\n    # Velocidade média do pistão (aproximação)\n    df_features['Mean_Piston_Speed'] = df_features['RPM'] * 0.1  # simplificado\n    \n    # Densidade do ar (aproximação baseada em temp)\n    df_features['Air_Density_Factor'] = 1 / (df_features['INTAKE_TEMP'] + 273.15)  # Lei dos gases ideais\n    \n    # 7. Features de operação do motor\n    # Carga específica (load normalizado por RPM)\n    df_features['Specific_Load'] = df_features['FuelLoad'] / (df_features['RPM'] / 1000)\n    \n    # 8. One-hot encoding para features categóricas\n    df_encoded = pd.get_dummies(df_features, columns=['RPM_Range', 'Load_Range'], prefix=['RPM', 'Load'])\n    \n    # 9. Features de posição na tabela (normalized grid position)\n    if 'rpm_bins' in globals() and 'load_bins' in globals():\n        # Posição normalizada na grid\n        df_encoded['RPM_Grid_Position'] = np.interp(df_encoded['RPM'], rpm_bins, np.linspace(0, 1, len(rpm_bins)))\n        df_encoded['Load_Grid_Position'] = np.interp(df_encoded['FuelLoad'], load_bins, np.linspace(0, 1, len(load_bins)))\n    \n    # Lista final de features para o modelo\n    ml_features = [\n        # Features básicas\n        'RPM', 'FuelLoad', 'VE_Current', 'Lambda_Delayed', 'Lambda_Target',\n        'COOLANT_TEMP', 'INTAKE_TEMP', 'TIMING_ADVANCE',\n        \n        # Features de interação\n        'RPM_Load_Interaction', 'VE_Lambda_Interaction',\n        \n        # Features de tendência\n        'RPM_Trend', 'Load_Trend', 'Lambda_Trend',\n        \n        # Features de eficiência\n        'Lambda_Efficiency',\n        \n        # Features físicas\n        'Mean_Piston_Speed', 'Air_Density_Factor', 'Specific_Load'\n    ]\n    \n    # Adiciona features categóricas one-hot encoded\n    categorical_features = [col for col in df_encoded.columns if col.startswith(('RPM_', 'Load_'))]\n    ml_features.extend(categorical_features)\n    \n    # Adiciona features de posição se disponíveis\n    if 'RPM_Grid_Position' in df_encoded.columns:\n        ml_features.extend(['RPM_Grid_Position', 'Load_Grid_Position'])\n    \n    # Remove features que podem ter NaN\n    df_encoded = df_encoded.dropna(subset=ml_features)\n    \n    print(f\"✅ Features criadas: {len(ml_features)}\")\n    print(f\"📊 Dados para ML: {len(df_encoded)} amostras\")\n    \n    # Estatísticas das features\n    print(f\"\\n📈 Principais features criadas:\")\n    for feature in ml_features[:10]:  # Mostra primeiras 10\n        if feature in df_encoded.columns:\n            print(f\"  {feature}: {df_encoded[feature].mean():.3f} ± {df_encoded[feature].std():.3f}\")\n    \n    return df_encoded, ml_features\n\n# Visualização das features\ndef visualize_features(df_features, ml_features):\n    \"\"\"Visualiza distribuições e correlações das features\"\"\"\n    \n    # Seleciona features numéricas principais\n    numeric_features = [f for f in ml_features if f in df_features.columns and \n                       df_features[f].dtype in ['float64', 'int64']][:12]\n    \n    if len(numeric_features) > 0:\n        # Matriz de correlação\n        plt.figure(figsize=(15, 12))\n        \n        # Correlação entre features\n        plt.subplot(2, 2, 1)\n        corr_matrix = df_features[numeric_features].corr()\n        sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', center=0, square=True)\n        plt.title('Matriz de Correlação das Features')\n        \n        # Distribuições das principais features\n        plt.subplot(2, 2, 2)\n        df_features[['RPM', 'FuelLoad', 'VE_Current']].hist(bins=30, alpha=0.7)\n        plt.title('Distribuições das Features Principais')\n        \n        # Correlação com target (VE_Corrected)\n        plt.subplot(2, 2, 3)\n        if 'VE_Corrected' in df_features.columns:\n            correlations = []\n            for feature in numeric_features:\n                corr = df_features[feature].corr(df_features['VE_Corrected'])\n                correlations.append(corr)\n            \n            feature_names = [f.replace('_', '\\n') for f in numeric_features]\n            plt.barh(range(len(correlations)), correlations)\n            plt.yticks(range(len(correlations)), feature_names)\n            plt.xlabel('Correlação com VE_Corrected')\n            plt.title('Importância das Features')\n            plt.grid(True, alpha=0.3)\n        \n        # Feature importance plot (simplified)\n        plt.subplot(2, 2, 4)\n        # Variância das features (normalizada)\n        variances = []\n        for feature in numeric_features[:8]:  # Top 8\n            var = df_features[feature].var()\n            variances.append(var)\n        \n        # Normaliza variâncias\n        variances = np.array(variances)\n        variances = variances / variances.max()\n        \n        feature_names = [f.replace('_', '\\n') for f in numeric_features[:8]]\n        plt.barh(range(len(variances)), variances)\n        plt.yticks(range(len(variances)), feature_names)\n        plt.xlabel('Variância Normalizada')\n        plt.title('Variabilidade das Features')\n        plt.grid(True, alpha=0.3)\n        \n        plt.tight_layout()\n        plt.show()\n\n# Executa feature engineering\nif 'df_ve_analysis' in locals() and df_ve_analysis is not None:\n    df_ml_ready, feature_list = create_features_for_ml(df_ve_analysis)\n    \n    # Visualiza features\n    visualize_features(df_ml_ready, feature_list)\n    \n    print(f\"\\n🎯 Target variable: VE_Corrected\")\n    print(f\"📊 Target range: {df_ml_ready['VE_Corrected'].min():.1f}% - {df_ml_ready['VE_Corrected'].max():.1f}%\")\n    print(f\"📈 Target mean: {df_ml_ready['VE_Corrected'].mean():.1f}% ± {df_ml_ready['VE_Corrected'].std():.1f}%\")\nelse:\n    print(\"❌ Dados de análise VE não disponíveis para feature engineering\")\n    df_ml_ready, feature_list = None, []

## 6. Machine Learning Model Training

In [ ]:
# Treinamento de modelos de Machine Learning
def train_ve_prediction_models(df_ml, features, target='VE_Corrected'):
    \"\"\"Treina múltiplos modelos para predição de VE otimizado\"\"\"
    
    print(\"🤖 Treinando modelos de ML para predição de VE...\")\n    \n    # Prepara dados\n    X = df_ml[features].copy()\n    y = df_ml[target].copy()\n    \n    # Remove linhas com NaN\n    mask = ~(X.isnull().any(axis=1) | y.isnull())\n    X = X[mask]\n    y = y[mask]\n    \n    print(f\"📊 Dados de treinamento: {len(X)} amostras, {len(features)} features\")\n    \n    # Normalização das features\n    scaler = StandardScaler()\n    X_scaled = scaler.fit_transform(X)\n    X_scaled_df = pd.DataFrame(X_scaled, columns=features, index=X.index)\n    \n    # Split train/validation/test\n    X_temp, X_test, y_temp, y_test = train_test_split(X_scaled_df, y, test_size=0.2, random_state=42)\n    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)\n    \n    print(f\"📊 Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}\")\n    \n    # Modelos para testar\n    models = {\n        'Random Forest': RandomForestRegressor(\n            n_estimators=200, \n            max_depth=15, \n            min_samples_split=5,\n            min_samples_leaf=2,\n            random_state=42,\n            n_jobs=-1\n        ),\n        'Gradient Boosting': GradientBoostingRegressor(\n            n_estimators=200,\n            max_depth=8,\n            learning_rate=0.1,\n            subsample=0.8,\n            random_state=42\n        ),\n        'Linear Regression': LinearRegression()\n    }\n    \n    # Treina e avalia modelos\n    model_results = {}\n    best_model = None\n    best_score = float('inf')\n    \n    for name, model in models.items():\n        print(f\"\\n🔄 Treinando {name}...\")\n        \n        # Treina modelo\n        model.fit(X_train, y_train)\n        \n        # Predições\n        y_train_pred = model.predict(X_train)\n        y_val_pred = model.predict(X_val)\n        y_test_pred = model.predict(X_test)\n        \n        # Métricas\n        results = {\n            'model': model,\n            'train_mse': mean_squared_error(y_train, y_train_pred),\n            'val_mse': mean_squared_error(y_val, y_val_pred),\n            'test_mse': mean_squared_error(y_test, y_test_pred),\n            'train_r2': r2_score(y_train, y_train_pred),\n            'val_r2': r2_score(y_val, y_val_pred),\n            'test_r2': r2_score(y_test, y_test_pred),\n            'train_mae': mean_absolute_error(y_train, y_train_pred),\n            'val_mae': mean_absolute_error(y_val, y_val_pred),\n            'test_mae': mean_absolute_error(y_test, y_test_pred),\n            'predictions': {\n                'train': y_train_pred,\n                'val': y_val_pred,\n                'test': y_test_pred\n            }\n        }\n        \n        model_results[name] = results\n        \n        # Atualiza melhor modelo baseado em validation score\n        if results['val_mse'] < best_score:\n            best_score = results['val_mse']\n            best_model = (name, model)\n        \n        # Imprime resultados\n        print(f\"  Train - MSE: {results['train_mse']:.3f}, R²: {results['train_r2']:.3f}, MAE: {results['train_mae']:.3f}\")\n        print(f\"  Val   - MSE: {results['val_mse']:.3f}, R²: {results['val_r2']:.3f}, MAE: {results['val_mae']:.3f}\")\n        print(f\"  Test  - MSE: {results['test_mse']:.3f}, R²: {results['test_r2']:.3f}, MAE: {results['test_mae']:.3f}\")\n    \n    print(f\"\\n🏆 Melhor modelo: {best_model[0]} (Val MSE: {best_score:.3f})\")\n    \n    # Visualiza comparação dos modelos\n    plt.figure(figsize=(18, 12))\n    \n    # Métricas comparativas\n    plt.subplot(2, 3, 1)\n    model_names = list(model_results.keys())\n    val_r2_scores = [model_results[name]['val_r2'] for name in model_names]\n    colors = ['skyblue', 'lightgreen', 'lightcoral']\n    bars = plt.bar(model_names, val_r2_scores, color=colors)\n    plt.ylabel('R² Score (Validation)')\n    plt.title('Comparação R² dos Modelos')\n    plt.xticks(rotation=45)\n    # Adiciona valores nas barras\n    for bar, score in zip(bars, val_r2_scores):\n        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, \n                f'{score:.3f}', ha='center', va='bottom')\n    plt.grid(True, alpha=0.3)\n    \n    # MSE comparativo\n    plt.subplot(2, 3, 2)\n    val_mse_scores = [model_results[name]['val_mse'] for name in model_names]\n    bars = plt.bar(model_names, val_mse_scores, color=colors)\n    plt.ylabel('MSE (Validation)')\n    plt.title('Comparação MSE dos Modelos')\n    plt.xticks(rotation=45)\n    for bar, score in zip(bars, val_mse_scores):\n        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, \n                f'{score:.2f}', ha='center', va='bottom')\n    plt.grid(True, alpha=0.3)\n    \n    # Predições vs Real (melhor modelo)\n    best_name = best_model[0]\n    best_results = model_results[best_name]\n    \n    plt.subplot(2, 3, 3)\n    plt.scatter(y_test, best_results['predictions']['test'], alpha=0.6, s=10)\n    min_val = min(y_test.min(), best_results['predictions']['test'].min())\n    max_val = max(y_test.max(), best_results['predictions']['test'].max())\n    plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfeito')\n    plt.xlabel('VE Real (%)')\n    plt.ylabel('VE Predito (%)')\n    plt.title(f'Predições vs Real - {best_name}')\n    plt.legend()\n    plt.grid(True, alpha=0.3)\n    \n    # Resíduos\n    plt.subplot(2, 3, 4)\n    residuals = y_test - best_results['predictions']['test']\n    plt.scatter(best_results['predictions']['test'], residuals, alpha=0.6, s=10)\n    plt.axhline(y=0, color='red', linestyle='--', alpha=0.7)\n    plt.xlabel('VE Predito (%)')\n    plt.ylabel('Resíduos')\n    plt.title(f'Análise de Resíduos - {best_name}')\n    plt.grid(True, alpha=0.3)\n    \n    # Feature importance (Random Forest)\n    if 'Random Forest' in model_results:\n        rf_model = model_results['Random Forest']['model']\n        feature_importance = rf_model.feature_importances_\n        \n        plt.subplot(2, 3, 5)\n        # Top 10 features mais importantes\n        top_indices = np.argsort(feature_importance)[-10:]\n        top_features = [features[i] for i in top_indices]\n        top_importance = feature_importance[top_indices]\n        \n        plt.barh(range(len(top_features)), top_importance)\n        plt.yticks(range(len(top_features)), [f.replace('_', '\\n') for f in top_features])\n        plt.xlabel('Importância')\n        plt.title('Top 10 Features Importantes (RF)')\n        plt.grid(True, alpha=0.3)\n    \n    # Distribuição dos erros\n    plt.subplot(2, 3, 6)\n    plt.hist(residuals, bins=30, alpha=0.7, edgecolor='black')\n    plt.axvline(residuals.mean(), color='red', linestyle='--', alpha=0.7, \n               label=f'Média: {residuals.mean():.2f}')\n    plt.xlabel('Resíduos')\n    plt.ylabel('Frequência')\n    plt.title('Distribuição dos Resíduos')\n    plt.legend()\n    plt.grid(True, alpha=0.3)\n    \n    plt.tight_layout()\n    plt.show()\n    \n    return model_results, scaler, (X_train, X_val, X_test, y_train, y_val, y_test), best_model\n\n# Executa treinamento\nif df_ml_ready is not None and len(feature_list) > 0:\n    model_results, scaler, data_splits, best_model = train_ve_prediction_models(df_ml_ready, feature_list)\n    \n    print(f\"\\n✅ Treinamento concluído!\")\n    print(f\"🎯 Melhor modelo: {best_model[0]}\")\n    print(f\"📊 Features utilizadas: {len(feature_list)}\")\nelse:\n    print(\"❌ Dados não disponíveis para treinamento\")\n    model_results, scaler, data_splits, best_model = None, None, None, None

## 7. VE Adjustment Prediction Function

In [ ]:
# Função de predição de ajustes VE\nclass VE_Optimizer:\n    \"\"\"Classe para otimização de VE usando modelo treinado\"\"\"\n    \n    def __init__(self, model, scaler, feature_list, rpm_bins, load_bins, ve_table):\n        self.model = model\n        self.scaler = scaler\n        self.feature_list = feature_list\n        self.rpm_bins = rpm_bins\n        self.load_bins = load_bins\n        self.ve_table = ve_table\n    \n    def predict_ve_adjustment(self, rpm, fuel_load, lambda_measured, \n                            coolant_temp=90, intake_temp=25, timing_advance=25):\n        \"\"\"Prediz ajuste VE para condições específicas\"\"\"\n        \n        # Calcula VE atual da tabela\n        current_ve = self._interpolate_ve(rpm, fuel_load)\n        \n        # Calcula lambda target\n        lambda_target = self._calculate_lambda_target(rpm, fuel_load)\n        \n        # Cria features para predição\n        features_dict = {\n            'RPM': rpm,\n            'FuelLoad': fuel_load,\n            'VE_Current': current_ve,\n            'Lambda_Delayed': lambda_measured,\n            'Lambda_Target': lambda_target,\n            'COOLANT_TEMP': coolant_temp,\n            'INTAKE_TEMP': intake_temp,\n            'TIMING_ADVANCE': timing_advance\n        }\n        \n        # Features derivadas\n        features_dict['RPM_Load_Interaction'] = rpm * fuel_load / 1000\n        features_dict['VE_Lambda_Interaction'] = current_ve * lambda_measured\n        features_dict['RPM_Trend'] = rpm  # Simplificado\n        features_dict['Load_Trend'] = fuel_load\n        features_dict['Lambda_Trend'] = lambda_measured\n        features_dict['Lambda_Efficiency'] = 1 - abs(lambda_measured - lambda_target) / lambda_target\n        features_dict['Mean_Piston_Speed'] = rpm * 0.1\n        features_dict['Air_Density_Factor'] = 1 / (intake_temp + 273.15)\n        features_dict['Specific_Load'] = fuel_load / (rpm / 1000)\n        \n        # Features categóricas (simplificado)\n        features_dict.update({\n            'RPM_Idle': 1 if rpm < 1500 else 0,\n            'RPM_Low': 1 if 1500 <= rpm < 3000 else 0,\n            'RPM_Mid': 1 if 3000 <= rpm < 5000 else 0,\n            'RPM_High': 1 if rpm >= 5000 else 0,\n            'Load_Light': 1 if fuel_load < 30 else 0,\n            'Load_Moderate': 1 if 30 <= fuel_load < 70 else 0,\n            'Load_Heavy': 1 if 70 <= fuel_load < 100 else 0,\n            'Load_WOT': 1 if fuel_load >= 100 else 0\n        })\n        \n        # Features de posição na grid\n        if self.rpm_bins is not None and self.load_bins is not None:\n            features_dict['RPM_Grid_Position'] = np.interp(rpm, self.rpm_bins, np.linspace(0, 1, len(self.rpm_bins)))\n            features_dict['Load_Grid_Position'] = np.interp(fuel_load, self.load_bins, np.linspace(0, 1, len(self.load_bins)))\n        \n        # Cria array de features na ordem correta\n        feature_values = []\n        for feature in self.feature_list:\n            if feature in features_dict:\n                feature_values.append(features_dict[feature])\n            else:\n                feature_values.append(0)  # Valor padrão para features ausentes\n        \n        # Normaliza features\n        features_array = np.array(feature_values).reshape(1, -1)\n        features_scaled = self.scaler.transform(features_array)\n        \n        # Predição\n        predicted_ve = self.model.predict(features_scaled)[0]\n        \n        # Aplicar limites de segurança (±25% do valor original)\n        min_ve = current_ve * 0.75\n        max_ve = current_ve * 1.25\n        predicted_ve = np.clip(predicted_ve, min_ve, max_ve)\n        \n        # Calcula estatísticas\n        correction_factor = predicted_ve / current_ve\n        percent_change = (correction_factor - 1) * 100\n        lambda_error = lambda_measured - lambda_target\n        \n        return {\n            'current_ve': current_ve,\n            'predicted_ve': predicted_ve,\n            'correction_factor': correction_factor,\n            'percent_change': percent_change,\n            'lambda_measured': lambda_measured,\n            'lambda_target': lambda_target,\n            'lambda_error': lambda_error,\n            'safety_limited': predicted_ve != self.model.predict(features_scaled)[0]\n        }\n    \n    def _interpolate_ve(self, rpm, fuel_load):\n        \"\"\"Interpola VE da tabela atual\"\"\"\n        if self.ve_table is None:\n            return 50.0\n        \n        interp_func = interpolate.interp2d(self.rpm_bins, self.load_bins, self.ve_table, kind='linear')\n        rpm_clipped = np.clip(rpm, min(self.rpm_bins), max(self.rpm_bins))\n        load_clipped = np.clip(fuel_load, min(self.load_bins), max(self.load_bins))\n        \n        return float(interp_func(rpm_clipped, load_clipped)[0])\n    \n    def _calculate_lambda_target(self, rpm, fuel_load):\n        \"\"\"Calcula lambda target baseado em condições\"\"\"\n        if fuel_load < 25:\n            return 1.0\n        elif fuel_load < 60:\n            return 0.95\n        elif fuel_load < 80:\n            return 0.88\n        else:\n            return 0.85\n    \n    def generate_optimized_table(self, test_conditions=None):\n        \"\"\"Gera nova tabela VE otimizada\"\"\"\n        \n        print(\"🔧 Gerando tabela VE otimizada...\")\n        \n        new_table = self.ve_table.copy()\n        adjustments_made = 0\n        \n        # Para cada célula da tabela\n        for i, load in enumerate(self.load_bins):\n            for j, rpm in enumerate(self.rpm_bins):\n                \n                # Simula leitura lambda (baseada no erro típico observado)\n                current_ve = self.ve_table[i, j]\n                lambda_target = self._calculate_lambda_target(rpm, load)\n                \n                # Simula lambda atual (com erro típico)\n                # Em uma implementação real, isso viria dos dados de log\n                lambda_measured = lambda_target + np.random.normal(0, 0.05)  # Erro simulado\n                \n                # Prediz novo VE\n                prediction = self.predict_ve_adjustment(rpm, load, lambda_measured)\n                \n                # Aplica ajuste se significativo (>2%)\n                if abs(prediction['percent_change']) > 2.0:\n                    new_table[i, j] = prediction['predicted_ve']\n                    adjustments_made += 1\n        \n        total_cells = len(self.load_bins) * len(self.rpm_bins)\n        print(f\"📊 Ajustes aplicados: {adjustments_made}/{total_cells} células ({adjustments_made/total_cells*100:.1f}%)\")\n        \n        return new_table\n\n# Cria otimizador se modelo estiver disponível\nif best_model is not None and scaler is not None:\n    optimizer = VE_Optimizer(\n        model=best_model[1],\n        scaler=scaler,\n        feature_list=feature_list,\n        rpm_bins=rpm_bins,\n        load_bins=load_bins,\n        ve_table=ve_table\n    )\n    \n    print(\"✅ Otimizador VE criado com sucesso!\")\n    \n    # Exemplo de uso\n    print(\"\\n🧪 Testando predição de VE:\")\n    \n    test_cases = [\n        {'rpm': 2000, 'fuel_load': 40, 'lambda_measured': 0.98, 'description': 'Cruise (lambda rico)'},\n        {'rpm': 3500, 'fuel_load': 75, 'lambda_measured': 0.91, 'description': 'Aceleração (lambda OK)'},\n        {'rpm': 5000, 'fuel_load': 95, 'lambda_measured': 0.82, 'description': 'WOT (lambda muito rico)'},\n        {'rpm': 1200, 'fuel_load': 20, 'lambda_measured': 1.05, 'description': 'Marcha lenta (lambda pobre)'}\n    ]\n    \n    for i, test in enumerate(test_cases, 1):\n        result = optimizer.predict_ve_adjustment(\n            rpm=test['rpm'],\n            fuel_load=test['fuel_load'],\n            lambda_measured=test['lambda_measured']\n        )\n        \n        print(f\"\\n  Teste {i}: {test['description']}\")\n        print(f\"    RPM: {test['rpm']}, Load: {test['fuel_load']} kPa\")\n        print(f\"    Lambda: {test['lambda_measured']:.3f} (target: {result['lambda_target']:.3f})\")\n        print(f\"    VE atual: {result['current_ve']:.1f}%\")\n        print(f\"    VE predito: {result['predicted_ve']:.1f}%\")\n        print(f\"    Mudança: {result['percent_change']:+.1f}%\")\n        if result['safety_limited']:\n            print(f\"    ⚠️  Limitado por segurança\")\nelse:\n    print(\"❌ Modelo não disponível para criar otimizador\")\n    optimizer = None

## 8. Results Visualization and Validation

In [ ]:
# Visualização e validação dos resultados finais\nif optimizer is not None:\n    print(\"🎨 Gerando visualizações finais e validação...\")\n    \n    # Gera tabela VE otimizada\n    optimized_table = optimizer.generate_optimized_table()\n    \n    # Visualizações comparativas\n    fig, axes = plt.subplots(2, 3, figsize=(22, 14))\n    \n    # 1. Tabela VE Original\n    im1 = axes[0,0].imshow(ve_table, aspect='auto', origin='lower', cmap='RdYlBu_r', vmin=0, vmax=100)\n    axes[0,0].set_title('Tabela VE Original', fontsize=14, fontweight='bold')\n    axes[0,0].set_xlabel('RPM Bins')\n    axes[0,0].set_ylabel('Load Bins (kPa)')\n    \n    # Configurar ticks\n    rpm_ticks = np.arange(0, len(rpm_bins), 2)\n    load_ticks = np.arange(0, len(load_bins), 2)\n    axes[0,0].set_xticks(rpm_ticks)\n    axes[0,0].set_xticklabels([f'{rpm_bins[i]:.0f}' for i in rpm_ticks], rotation=45)\n    axes[0,0].set_yticks(load_ticks)\n    axes[0,0].set_yticklabels([f'{load_bins[i]:.0f}' for i in load_ticks])\n    \n    cbar1 = plt.colorbar(im1, ax=axes[0,0], shrink=0.8)\n    cbar1.set_label('VE (%)', rotation=270, labelpad=15)\n    \n    # 2. Tabela VE Otimizada\n    im2 = axes[0,1].imshow(optimized_table, aspect='auto', origin='lower', cmap='RdYlBu_r', vmin=0, vmax=100)\n    axes[0,1].set_title('Tabela VE Otimizada (IA)', fontsize=14, fontweight='bold')\n    axes[0,1].set_xlabel('RPM Bins')\n    axes[0,1].set_ylabel('Load Bins (kPa)')\n    axes[0,1].set_xticks(rpm_ticks)\n    axes[0,1].set_xticklabels([f'{rpm_bins[i]:.0f}' for i in rpm_ticks], rotation=45)\n    axes[0,1].set_yticks(load_ticks)\n    axes[0,1].set_yticklabels([f'{load_bins[i]:.0f}' for i in load_ticks])\n    \n    cbar2 = plt.colorbar(im2, ax=axes[0,1], shrink=0.8)\n    cbar2.set_label('VE (%)', rotation=270, labelpad=15)\n    \n    # 3. Diferença (Otimizada - Original)\n    diff_table = optimized_table - ve_table\n    im3 = axes[0,2].imshow(diff_table, aspect='auto', origin='lower', cmap='RdBu_r', \n                          vmin=-10, vmax=10)\n    axes[0,2].set_title('Diferença (Otimizada - Original)', fontsize=14, fontweight='bold')\n    axes[0,2].set_xlabel('RPM Bins')\n    axes[0,2].set_ylabel('Load Bins (kPa)')\n    axes[0,2].set_xticks(rpm_ticks)\n    axes[0,2].set_xticklabels([f'{rpm_bins[i]:.0f}' for i in rpm_ticks], rotation=45)\n    axes[0,2].set_yticks(load_ticks)\n    axes[0,2].set_yticklabels([f'{load_bins[i]:.0f}' for i in load_ticks])\n    \n    cbar3 = plt.colorbar(im3, ax=axes[0,2], shrink=0.8)\n    cbar3.set_label('Δ VE (%)', rotation=270, labelpad=15)\n    \n    # 4. Distribuição das mudanças\n    axes[1,0].hist(diff_table.flatten(), bins=30, alpha=0.7, edgecolor='black', color='skyblue')\n    axes[1,0].axvline(0, color='red', linestyle='--', alpha=0.7, linewidth=2)\n    axes[1,0].set_xlabel('Mudança VE (%)')\n    axes[1,0].set_ylabel('Frequência')\n    axes[1,0].set_title('Distribuição das Mudanças VE')\n    axes[1,0].grid(True, alpha=0.3)\n    \n    # Estatísticas\n    mean_change = diff_table.mean()\n    std_change = diff_table.std()\n    max_increase = diff_table.max()\n    max_decrease = diff_table.min()\n    \n    axes[1,0].text(0.02, 0.98, f'Média: {mean_change:.2f}%\\nStd: {std_change:.2f}%\\nMax+: {max_increase:.1f}%\\nMax-: {max_decrease:.1f}%',\n                  transform=axes[1,0].transAxes, verticalalignment='top',\n                  bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))\n    \n    # 5. Análise por zona de RPM\n    rpm_zones = {\n        'Baixo (500-2000)': (0, 6),\n        'Médio (2000-4000)': (6, 12),\n        'Alto (4000+)': (12, 16)\n    }\n    \n    zone_changes = []\n    zone_labels = []\n    \n    for zone_name, (start_idx, end_idx) in rpm_zones.items():\n        zone_diff = diff_table[:, start_idx:end_idx]\n        zone_changes.extend(zone_diff.flatten())\n        zone_labels.extend([zone_name] * zone_diff.size)\n    \n    # Boxplot por zona\n    zone_data = []\n    for zone_name, (start_idx, end_idx) in rpm_zones.items():\n        zone_diff = diff_table[:, start_idx:end_idx]\n        zone_data.append(zone_diff.flatten())\n    \n    bp = axes[1,1].boxplot(zone_data, labels=list(rpm_zones.keys()), patch_artist=True)\n    axes[1,1].set_ylabel('Mudança VE (%)')\n    axes[1,1].set_title('Mudanças VE por Zona de RPM')\n    axes[1,1].grid(True, alpha=0.3)\n    axes[1,1].axhline(0, color='red', linestyle='--', alpha=0.7)\n    \n    # Cores para boxplot\n    colors = ['lightblue', 'lightgreen', 'lightcoral']\n    for patch, color in zip(bp['boxes'], colors):\n        patch.set_facecolor(color)\n        patch.set_alpha(0.7)\n    \n    # 6. Mapa de confiança (baseado na densidade de dados do log)\n    if 'df_ml_ready' in locals() and df_ml_ready is not None:\n        # Cria grid de confiança baseado na quantidade de dados\n        confidence_map = np.zeros_like(ve_table)\n        \n        for i, load in enumerate(load_bins):\n            for j, rpm in enumerate(rpm_bins):\n                # Conta pontos próximos a esta célula\n                nearby_points = df_ml_ready[\n                    (abs(df_ml_ready['RPM'] - rpm) < 200) & \n                    (abs(df_ml_ready['FuelLoad'] - load) < 10)\n                ]\n                \n                # Confiança baseada no número de pontos\n                confidence_map[i, j] = min(len(nearby_points) / 50, 1.0)  # Normaliza para 0-1\n        \n        im6 = axes[1,2].imshow(confidence_map, aspect='auto', origin='lower', \n                              cmap='Greens', vmin=0, vmax=1)\n        axes[1,2].set_title('Mapa de Confiança\\n(baseado em densidade de dados)', fontsize=14, fontweight='bold')\n        axes[1,2].set_xlabel('RPM Bins')\n        axes[1,2].set_ylabel('Load Bins (kPa)')\n        axes[1,2].set_xticks(rpm_ticks)\n        axes[1,2].set_xticklabels([f'{rpm_bins[i]:.0f}' for i in rpm_ticks], rotation=45)\n        axes[1,2].set_yticks(load_ticks)\n        axes[1,2].set_yticklabels([f'{load_bins[i]:.0f}' for i in load_ticks])\n        \n        cbar6 = plt.colorbar(im6, ax=axes[1,2], shrink=0.8)\n        cbar6.set_label('Confiança', rotation=270, labelpad=15)\n    else:\n        # Se não tiver dados, mostra placeholder\n        axes[1,2].text(0.5, 0.5, 'Mapa de Confiança\\n(dados não disponíveis)', \n                      ha='center', va='center', transform=axes[1,2].transAxes,\n                      fontsize=12, bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.5))\n        axes[1,2].set_title('Mapa de Confiança')\n    \n    plt.tight_layout()\n    plt.show()\n    \n    # Relatório final\n    print(\"\\n\" + \"=\"*60)\n    print(\"📊 RELATÓRIO FINAL DE OTIMIZAÇÃO VE\")\n    print(\"=\"*60)\n    \n    total_cells = ve_table.size\n    changed_cells = np.sum(np.abs(diff_table) > 0.1)  # Mudanças > 0.1%\n    significant_changes = np.sum(np.abs(diff_table) > 2.0)  # Mudanças > 2%\n    \n    print(f\"🔢 Total de células na tabela: {total_cells}\")\n    print(f\"📈 Células modificadas (>0.1%): {changed_cells} ({changed_cells/total_cells*100:.1f}%)\")\n    print(f\"⚡ Mudanças significativas (>2%): {significant_changes} ({significant_changes/total_cells*100:.1f}%)\")\n    print(f\"📊 Mudança média: {mean_change:.2f}% ± {std_change:.2f}%\")\n    print(f\"📈 Maior aumento: +{max_increase:.1f}%\")\n    print(f\"📉 Maior redução: {max_decrease:.1f}%\")\n    \n    # Análise por zona de carga\n    print(f\"\\n🎯 Análise por zona de carga:\")\n    load_zones = [\n        ('Baixa (0-40 kPa)', 0, 7),\n        ('Média (40-70 kPa)', 7, 13),\n        ('Alta (70+ kPa)', 13, 16)\n    ]\n    \n    for zone_name, start_idx, end_idx in load_zones:\n        zone_diff = diff_table[start_idx:end_idx, :]\n        zone_mean = zone_diff.mean()\n        zone_std = zone_diff.std()\n        print(f\"  {zone_name}: {zone_mean:+.2f}% ± {zone_std:.2f}%\")\n    \n    print(f\"\\n🤖 Modelo utilizado: {best_model[0]}\")\n    if 'estimated_delay' in locals():\n        print(f\"⏱️  Delay da sonda estimado: {estimated_delay:.2f}s\")\n    \n    print(f\"\\n💾 Para aplicar esta tabela otimizada:\")\n    print(f\"   1. Revise as mudanças significativas\")\n    print(f\"   2. Teste em condições controladas\")\n    print(f\"   3. Monitore lambda em tempo real\")\n    print(f\"   4. Faça ajustes incrementais se necessário\")\n    \n    print(f\"\\n⚠️  IMPORTANTE: Sempre monitore temperatura dos gases de escape\")\n    print(f\"    e parâmetros de detonação ao aplicar mudanças significativas!\")\n    \nelse:\n    print(\"❌ Otimizador não disponível para gerar resultados finais\")\n\nprint(\"\\n\" + \"=\"*60)\nprint(\"✅ ANÁLISE DE OTIMIZAÇÃO VE CONCLUÍDA!\")\nprint(\"=\"*60)